In [1]:
import os
import uproot
import pandas as pd
import time
from datetime import timedelta
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score

In [2]:
def load_large_rootfile_to_df(rootfile, columns=None, tree="sel", chunksize=100_000):
    """
    Load a large ROOT file into a pandas DataFrame while optimizing memory usage.

    Parameters:
    file_path (str): The path to the ROOT file.
    tree_name (str): The name of the TTree to load.
    columns (list, optional): A list of columns to load. If None, all columns will be loaded.
    chunksize (int, optional): The number of rows to load in each chunk.

    Returns:
    A pandas DataFrame containing the data from the TTree.
    """
    df_list = []
    
    print(f"Loading the ROOT file: {rootfile}")
    ctime = time.time()
    
    # Open the ROOT file and get the TTree object
    with uproot.open(rootfile) as f:

        # Specify the columns to load
        if columns is None:
            columns = f[tree].keys()

        # Load the data in chunks
        for i in tqdm(range(0, f[tree].num_entries, chunksize)):
            df = f["sel"].arrays(columns, library="pd", entry_start=i, entry_stop=i+chunksize)
            
            # Append the chunk to the list
            df_list.append(df)
    
    print(f"ROOT file imported as a Dataframe in: {timedelta(seconds=time.time()-ctime)}")
    # Concatenate the chunks into a single DataFrame
    return pd.concat(df_list, ignore_index=True)


In [3]:
def resolution_function(
    true_value,
    reco_value,
):
    """
    Function to fit the resolution of the reconstruction software.
    
    Parameters:
    true_value (float): The true value of the parameter.
    reco_value (float): The reconstructed value of the parameter.

    Returns:
    The resolution of the reconstruction software.
    """

    return np.abs(reco_value - true_value) / true_value

In [4]:
def hyperbolic(
    x,
    a,
    b,
    c,
    d,
):
    return a / (b * x + c) + d

In [5]:
if __name__ == "__main__":
    # Define the path to the ROOT file
    #path = "/sps/km3net/users/mchadoli/ANTARES/mc/cut_selection/100GeV"
    #root_file = "full_nutau_low_updated.root"
    
    path = "/sps/km3net/users/mchadoli/Swim/Data/events/ANTARES"
    root_file = "antares_w_nnfit_FINAL1.root"
    COLUMNS = [
        "interaction_type", 
        "is_cc", "type",
        "energy_true",
        "cos_zenith_true",
        "NNFitTrack_Log10Energy",
        "NNFitShower_Log10Energy", 
        "NNFitShower_Theta",
        "NNFitTrack_Theta",
        ]

    # Load the ROOT file into a DataFrame
    df = load_large_rootfile_to_df(os.path.join(path, root_file), columns=COLUMNS)

    # Drop rows with Nan Values in NNFitShower
    df = df.dropna(subset=["NNFitShower_Log10Energy"])



In [6]:
df["theta_true"] = 180 - (np.arccos(df["cos_zenith_true"]) * 180 / np.pi) 

In [7]:
# Convert the log10 energy to energy
df["NNFitShower_Energy"] = 10**df["NNFitShower_Log10Energy"]
df["NNFitTrack_Energy"] = 10**df["NNFitTrack_Log10Energy"]

# Convert theta to cos(zenith)
df["NNFitShower_cos_zenith"] = - np.cos( np.radians(df["NNFitShower_Theta"]))
df["NNFitTrack_cos_zenith"] = - np.cos( np.radians(df["NNFitTrack_Theta"]))

## Energy Resolution

In [8]:
def energy_resolution_fit(
    df,
):

    # Define energy bins
    energy_bins = np.round(np.geomspace(10, 100, 15), 4)
    #energy_bins = np.append(energy_bins, 1e3)

    df["energy_bins"] = pd.cut(df["energy_true"], bins= energy_bins)

    energy_resolutions = df.groupby("energy_bins").apply(
        lambda x: pd.Series(
            {
                "energy_resolution_median": x["EnergyResolution"].median(),
                "energy_resolution_std": x["EnergyResolution"].std(),
            }
        )   
    ).reset_index()


    energy_resolutions["midpoint"] = energy_resolutions["energy_bins"].apply(lambda x: x.mid)

    return energy_resolutions


In [9]:
# Fit the energy resolutionax.set_ylabel("Energy resolution")
df["EnergyResolution"] = resolution_function(df["energy_true"], df["NNFitShower_Energy"])
energy_resolutions = energy_resolution_fit(df)

binning = np.linspace(0, 30, 50)

ax, fig = plt.subplots(figsize=(12, 6))
df.hist(
    column="EnergyResolution",
    bins=binning,
    alpha=0.5,
    label="Energy resolution",
    ax=fig,
    log=True,
)

In [10]:
# Fit the energy resolution
popt, pcov = curve_fit(
    hyperbolic,
    xdata = energy_resolutions["midpoint"].astype(float),
    ydata = energy_resolutions["energy_resolution_median"],
    sigma = energy_resolutions["energy_resolution_std"],
)
print("Error:", np.sqrt(np.diag(pcov)))

y_pred = hyperbolic(energy_resolutions["midpoint"].astype(float), *popt)

print("R2 score:", r2_score(energy_resolutions["energy_resolution_median"], y_pred))

energy_fit_params = pd.DataFrame(
    {
        "parameters": popt,
        "error": np.sqrt(np.diag(pcov)),
    })

In [21]:
df_plot = df[df["EnergyResolution"] < 100]

# Apply topology function


fig, ax = plt.subplots(figsize=(12, 6))
ax.hist2d(
    df_plot["energy_true"],
    df_plot["EnergyResolution"],
    bins=(50, 75),
    cmap="viridis",
    cmin=1,
)
ax.errorbar(
    energy_resolutions["midpoint"],
    energy_resolutions["energy_resolution_median"],
    yerr=energy_resolutions["energy_resolution_std"],
    fmt="o",
    color="black",
    label="median & std",
)
ax.plot(
    energy_resolutions["midpoint"].astype(float),
    hyperbolic(energy_resolutions["midpoint"].astype(float), *energy_fit_params["parameters"].values ),
    "--",
    color="red",
    label="Hyperbolic fit",
)
ax.set_ylim(0, 20)
ax.set_xlim(8, 100)
ax.legend()
ax.set_xlabel("True energy (GeV)")
ax.set_ylabel("Energy resolution")
plt.show()

## Direction Resolution

In [22]:
# Calculate the direction resolution
df["DirectionResolution"] = resolution_function(df["theta_true"], df["NNFitShower_Theta"])
df["DirectionResolution"] = df["DirectionResolution"].replace([np.inf, -np.inf], np.nan)
df.dropna(subset=["DirectionResolution"], inplace=True)
df = df[df["DirectionResolution"] < 10]

direction_bins = np.arange(-1, -0.6, 0.05)
direction_bins = np.append(direction_bins, np.linspace(-0.6, 0, 7))

df["direction_bins"] = pd.cut(df["cos_zenith_true"], bins= direction_bins)

direction_resolutions = df.groupby("direction_bins").apply(
    lambda x: pd.Series(
        {
            "dir_resolution_mean": x["DirectionResolution"].mean(),
            "dir_resolution_std": x["DirectionResolution"].std(),
        }
    )
).reset_index()

direction_resolutions["midpoint"] = direction_resolutions["direction_bins"].apply(lambda x: x.mid)

# Fit the direction resolution
popt, pcov = curve_fit(
    hyperbolic,
    xdata = direction_resolutions["midpoint"].astype(float),
    ydata = direction_resolutions["dir_resolution_mean"],
    sigma = direction_resolutions["dir_resolution_std"],
)

print("Error:", np.sqrt(np.diag(pcov)))
print("R2 score:", r2_score(direction_resolutions["dir_resolution_mean"], hyperbolic(direction_resolutions["midpoint"].astype(float), *popt)))

direction_fit_params = pd.DataFrame(
    {
        "parameters": popt,
        "error": np.sqrt(np.diag(pcov)),
    })

In [26]:
binning_y = np.linspace(0, 5, 150)
binning_x = np.linspace(-1, 0, 150)
x_data_dir = np.linspace(-0.975, 0, 100)


fig, ax = plt.subplots(figsize=(10, 6))
ax.hist2d(
    df["cos_zenith_true"],
    df["DirectionResolution"],
    bins= (binning_x, binning_y),
    cmap="viridis",
    cmin=10,
)
ax.errorbar(
    direction_resolutions["midpoint"],
    direction_resolutions["dir_resolution_mean"],
    yerr=direction_resolutions["dir_resolution_std"],
    fmt="o",
    color="black",
    label="mean & std",
)
ax.plot(
    x_data_dir,
    hyperbolic(x_data_dir, *direction_fit_params["parameters"].values),
    "--",
    color="red",
    label="Hyperbolic fit",
)
ax.legend()
plt.show()  

## Resolution as function of energy & direction

In [27]:
# Plot the energy resolution as a function of the true energy and the zenith angle
df = df[df["EnergyResolution"] < 20]
x_data_energy = np.linspace(10, 100, 100)

fig, ax = plt.subplots(1, 2, figsize=(16, 6))
ax[0].hist2d(
    df["energy_true"],
    df["EnergyResolution"],
    bins=(50, 50),
    cmap="viridis",
    cmin=1,
)
ax[0].errorbar(
    energy_resolutions["midpoint"],
    energy_resolutions["energy_resolution_median"],
    yerr=energy_resolutions["energy_resolution_std"],
    fmt="o",
    color="black",
    label="median & std",
)
ax[0].plot(
    x_data_energy,
    hyperbolic(x_data_energy, *energy_fit_params["parameters"].values),
    "--",
    color="red",
    label="Hyperbolic fit",
)

ax[0].set_xlabel("True energy (GeV)")
ax[0].set_ylabel("Energy resolution")
ax[0].set_title("Energy resolution as a function of the true energy")
ax[1].hist2d(
    df["cos_zenith_true"],
    df["EnergyResolution"],
    bins=(50, 50),
    cmap="viridis",
    cmin=1,
)
ax[1].set_xlabel("$\cos(\\theta)$")
ax[1].set_ylabel("Energy resolution")
ax[1].set_title("Energy resolution as a function of the zenith angle")
ax[0].legend()
fig.tight_layout()
fig.savefig("../plots/energy_resolution.png")

In [28]:
# Plot the direction resolution as a function of the true energy and the zenith angle
df = df[df["DirectionResolution"] < 5]
df = df[df["cos_zenith_true"] < 0]

fig, ax = plt.subplots(1, 2, figsize=(16, 6))
ax[0].hist2d(
    df["energy_true"],
    df["DirectionResolution"],
    bins=(100, 100),
    cmap="viridis",
    cmin=10,
)
ax[1].hist2d(
    df["cos_zenith_true"],
    df["DirectionResolution"],
    bins=(100, 100),
    cmap="viridis",
    cmin=10,
)
ax[1].errorbar(
    direction_resolutions["midpoint"],
    direction_resolutions["dir_resolution_mean"],
    yerr=direction_resolutions["dir_resolution_std"],
    fmt="o",
    color="black",
    label="mean & std",
)
ax[1].plot(
    x_data_dir,
    hyperbolic(x_data_dir, *direction_fit_params["parameters"].values),
    "--",
    color="red",
    label="Hyperbolic fit",
)
ax[1].legend()
ax[0].set_xlabel("True energy (GeV)")
ax[0].set_ylabel("Direction resolution")
ax[0].set_title("Direction resolution as a function of the true energy")
ax[1].set_xlabel("$\cos(\\theta)$")
ax[1].set_ylabel("Direction resolution")
ax[1].set_title("Direction resolution as a function of the zenith angle")
fig.tight_layout()
fig.savefig("../plots/direction_resolution.png")

In [29]:
# Export the resolution parameters to a CSV file
with open("../plots/resolution_parameters.csv", "w") as f:
    f.write("Energy resolution\n")
    f.write("a, b, c, d\n")
    f.write(", ".join(map(str, energy_fit_params["parameters"].values)))
    f.write("\n")
    f.write("Direction resolution\n")
    f.write(", ".join(map(str, direction_fit_params["parameters"].values)))
    f.write("\n")